In [49]:
import pandas as pd
import json
import os
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from xgboost import XGBClassifier

In [50]:
data_path = "data"
save_path = "predictions"

In [51]:
train_set_path = os.path.join(data_path,"train_set.json")
test_set_path = os.path.join(data_path,"test_set.json")
artificial_train_set_path = os.path.join(data_path,"ori_pqaa.json")

In [52]:
df_train = pd.read_json(train_set_path).T
df_train.head()

,QUESTION,CONTEXTS,LABELS,MESHES,YEAR,reasoning_required_pred,reasoning_free_pred,final_decision,LONG_ANSWER
19156007,Can clinicians use the PHQ-9 to assess depress...,[To investigate whether the Patient Health Que...,"[PURPOSE, METHODS, RESULTS]","[Activities of Daily Living, Aged, Blindness, ...",2009,yes,yes,yes,"Our findings demonstrate that the PHQ-9, when ..."
25155638,Can routine offering of influenza vaccination ...,[Influenza vaccination remains below the feder...,"[BACKGROUND, OBJECTIVE, DESIGN, PARTICIPANTS, ...","[Adolescent, Adult, African Americans, Aged, C...",2014,yes,yes,yes,Improved office-based practices regarding infl...
15670262,Is gastric electrical stimulation superior to ...,[Severe upper gastrointestinal (GI) motor diso...,"[CONTEXT, OBJECTIVE, DESIGN, SETTING AND PATIE...","[Adult, Cost-Benefit Analysis, Electric Stimul...",2005,yes,yes,yes,We conclude that GES is more effective in impr...
17610439,Do hospitals provide lower quality care on wee...,[To examine the effect of a weekend hospitaliz...,"[OBJECTIVE, DATA SOURCES, STUDY DESIGN, PRINCI...","[Aged, Female, Health Expenditures, Health Sta...",2007,yes,yes,yes,Weekend hospitalization leads to delayed provi...
10811329,Are 99mTc leukocyte scintigraphy and SBFT stud...,[The goal of this retrospective study was to a...,"[OBJECTIVE, METHODS, RESULTS]","[Adolescent, Child, Colitis, Ulcerative, Colon...",2000,yes,yes,yes,Our results suggest that 99mTc-WBC is useful a...


In [53]:
df_test = pd.read_json(test_set_path).T
df_test.head()

,QUESTION,CONTEXTS,LABELS,MESHES,YEAR,reasoning_required_pred,reasoning_free_pred,final_decision,LONG_ANSWER
12377809,Is anorectal endosonography valuable in dysche...,[Dyschesia can be provoked by inappropriate de...,"[AIMS, METHODS, RESULTS]","[Adolescent, Adult, Aged, Aged, 80 and over, A...",2002,yes,yes,yes,Linear anorectal endosonography demonstrated i...
26163474,Is there a connection between sublingual varic...,[Sublingual varices have earlier been related ...,"[BACKGROUND, METHODS, RESULTS]","[Adult, Aged, Angina Pectoris, Atrial Fibrilla...",2015,yes,yes,yes,An association was found between sublingual va...
19100463,Is the affinity column-mediated immunoassay me...,[Tacrolimus is a potent immunosuppressive drug...,"[BACKGROUND, METHODS, RESULTS]","[Bilirubin, Chromatography, Affinity, Creatini...",2008,maybe,yes,yes,The ACMIA method used for a tacrolimus assay i...
18537964,Does a physician's specialty influence the rec...,[To determine the impact of a physician's spec...,"[AIMS, METHODS, RESULTS]","[Adult, Attitude of Health Personnel, Clinical...",2008,yes,yes,yes,Physicians appear to document more frequently ...
12913878,Locoregional opening of the rodent blood-brain...,[Nd:YAG laser-induced thermo therapy (LITT) of...,"[BACKGROUND AND OBJECTIVES, MATERIALS AND METH...","[Administration, Topical, Animals, Antineoplas...",2003,yes,yes,yes,LITT induces a locoregional passage of chemoth...


In [54]:
df_artificial_train = pd.read_json(artificial_train_set_path).T
df_artificial_train

,QUESTION,CONTEXTS,LABELS,LONG_ANSWER,MESHES,final_decision
25429730,Are group 2 innate lymphoid cells ( ILC2s ) in...,[Chronic rhinosinusitis (CRS) is a heterogeneo...,"[BACKGROUND, OBJECTIVE, METHODS, RESULTS]","As ILC2s are elevated in patients with CRSwNP,...","[Adult, Aged, Antigens, Surface, Case-Control ...",yes
25433161,Does vagus nerve contribute to the development...,[Phosphatidylethanolamine N-methyltransferase ...,"[OBJECTIVE, METHODS, RESULTS]",Neuronal signals via the hepatic vagus nerve c...,"[Animals, Chemokine CCL2, Diet, High-Fat, Dise...",yes
25445714,Does psammaplin A induce Sirtuin 1-dependent a...,[Psammaplin A (PsA) is a natural product isola...,"[BACKGROUND, METHODS]",PsA significantly inhibited MCF-7/adr cells pr...,"[Acetylation, Animals, Antibiotics, Antineopla...",yes
25431941,Is methylation of the FGFR2 gene associated wi...,[This study examined links between DNA methyla...,"[OBJECTIVE, METHODS, RESULTS]",We identified a novel biologically plausible c...,[],yes
25432519,Do tumor-infiltrating immune cell profiles and...,[Tumor microenvironment immunity is associated...,"[BACKGROUND, METHODS, RESULTS]",Breast cancer immune cell subpopulation profil...,"[Adult, Aged, Anthracyclines, Antibodies, Mono...",yes
...,...,...,...,...,...,...
8217974,Is urine production rate related to behavioura...,[To investigate the relation between hourly fe...,"[OBJECTIVE, METHODS, METHODS, METHODS, METHODS...",During active sleep (state 2F) hourly fetal ur...,"[Behavior, Embryonic and Fetal Development, Fe...",yes
8204319,Does evaluation of the use of general practice...,[This study set out to show how well samples f...,"[OBJECTIVE, METHODS, RESULTS]",General practice registers can provide a suita...,"[Adult, Age Factors, Epidemiologic Methods, Fa...",yes
8205673,Does intracoronary angiotensin-converting enzy...,[There is increasing recognition of myocardial...,"[BACKGROUND, RESULTS]",Intracoronary enalaprilat resulted in an impro...,"[Adult, Aged, Coronary Vessels, Diastole, Enal...",yes
8215873,Does transfusion significantly increase the ri...,[To determine if splenectomy results in an inc...,"[OBJECTIVE, METHODS, METHODS, METHODS, METHODS...",The choice between splenectomy and splenic rep...,"[Adult, Bacteremia, Female, Humans, Injury Sev...",yes


In [55]:
print(f"Train shape : {df_train.shape}")
print(f"Test shape : {df_test.shape}")
print(f"Artificial train shape : {df_artificial_train.shape}")


Train shape : (500, 9)
Test shape : (500, 9)
Artificial train shape : (211269, 6)


In [56]:
def save_preds(keys,values,file_name,pred_dir = save_path):

    pred_dict = {}
    file_path = os.path.join(pred_dir,file_name+".json")
    for key,value in zip(keys,values):
        pred_dict[key] = value
    
    with open(file_path,
              "w") as outfile:
        json.dump(pred_dict,outfile)


In [57]:
train_combined_texts = [f"Question: {q} Context: {c}" for q, c in zip(df_train['QUESTION'], df_train['CONTEXTS'])]
test_combined_texts = [f"Question: {q} Context: {c}" for q, c in zip(df_test['QUESTION'], df_test['CONTEXTS'])]
artificial_train_combined_texts = [f"Question: {q} Context: {c}" for q, c in zip(df_artificial_train['QUESTION'], df_artificial_train['CONTEXTS'])]


In [58]:
save_preds(df_test.index.to_list(),["yes"]*500,"mode_pred")

## Original train dataset

In [59]:
label_encoder = LabelEncoder()
vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=5000)

In [60]:
train_y = label_encoder.fit_transform(df_train['final_decision'])
train_x = vectorizer.fit_transform(train_combined_texts)

In [61]:
test_x = vectorizer.transform(test_combined_texts)

In [62]:
# clf = MultinomialNB()
clf = ComplementNB()
clf = clf.fit(train_x,train_y)

In [63]:
test_preds = clf.predict(test_x)

In [64]:
test_labels = label_encoder.inverse_transform(test_preds)

In [65]:
save_preds(df_test.index.to_list(),test_labels,"tfidf_complementNB_pred")

## Artificial train dataset

In [66]:
label_encoder_combined = LabelEncoder()
vectorizer_combined = TfidfVectorizer(ngram_range=(1,2),max_features=5000)

In [67]:
combined_y = df_train['final_decision'].to_list()
combined_y.extend(df_artificial_train['final_decision'].to_list())

In [68]:
len(combined_y)

211769

In [69]:

combined_x = train_combined_texts.copy()
combined_x.extend(artificial_train_combined_texts)



In [70]:
len(combined_x)

211769

In [71]:
train_combined_y = label_encoder_combined.fit_transform(combined_y)

In [72]:
train_combined_x = vectorizer_combined.fit_transform(combined_x)

In [73]:
test_x = vectorizer_combined.transform(test_combined_texts)

In [74]:
clf = ComplementNB()
# clf = MultinomialNB()
# clf = XGBClassifier()
clf = clf.fit(train_combined_x,train_combined_y
)

# sample_weight=compute_sample_weight("balanced",train_combined_y

In [75]:
test_preds = clf.predict(test_x)
test_labels = label_encoder_combined.inverse_transform(test_preds)
save_preds(df_test.index.to_list(),test_labels,"tfidf_complementNB_combined_pred")

In [76]:
np.unique(train_combined_y, return_counts=True)

(array([0, 1, 2]), array([    55,  15294, 196420]))

In [77]:
label_encoder_combined.classes_

array(['maybe', 'no', 'yes'], dtype='<U5')